# **트위터 '배민커넥트' 끌어오기**

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

### 1. 트위터 API로 데이터 가져오기

In [36]:
import tweepy

api_key = "RvnZeIl8ra88reu8fm23m0bST"
api_secret = "wTRylK94GK2KmhZUnqXonDaIszwAsS6VPvpSsIo6EX5GQLtzQo"
access_token = "959614462004117506-dkWyZaO8Bz3ZXh73rspWfc1sQz0EnDU"
access_token_secret = "rxDWfg7uz1yXMTDwijz0x90yWhDAnmOM15R6IgC8kmtTe"

auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_token_secret)

# twitter API를 사용하기 위한 준비
api = tweepy.API(auth)

In [37]:
# 크롤링된 데이터를 저장할 데이터 프레임
# 15분에 한번씩만 가능

columns = ['created', 'tweet_text']
df = pd.DataFrame(columns=columns)

keyword='식었음'

# twitter API를 사용하여 ‘쿠리어’이 포함된 100페이지의 트윗들을 크롤링한 뒤, ‘text’, ‘created_at’ 정보를 데이터 프레임으로 저장
for i in range(1,100):
    print("Get data", str(i/500*100), "% complete..")
    tweets = api.search(keyword)
    for tweet in tweets:
        tweet_text = tweet.text
        created = tweet.created_at
        row = [created, tweet_text]
        series = pd.Series(row, index=df.columns)
        df = df.append(series, ignore_index=True)
print("Get data 100 % complete..")

Get data 0.2 % complete..
Get data 0.4 % complete..
Get data 0.6 % complete..
Get data 0.8 % complete..
Get data 1.0 % complete..
Get data 1.2 % complete..
Get data 1.4000000000000001 % complete..
Get data 1.6 % complete..
Get data 1.7999999999999998 % complete..
Get data 2.0 % complete..
Get data 2.1999999999999997 % complete..
Get data 2.4 % complete..
Get data 2.6 % complete..
Get data 2.8000000000000003 % complete..
Get data 3.0 % complete..
Get data 3.2 % complete..
Get data 3.4000000000000004 % complete..
Get data 3.5999999999999996 % complete..
Get data 3.8 % complete..
Get data 4.0 % complete..
Get data 4.2 % complete..
Get data 4.3999999999999995 % complete..
Get data 4.6 % complete..
Get data 4.8 % complete..
Get data 5.0 % complete..
Get data 5.2 % complete..
Get data 5.4 % complete..
Get data 5.6000000000000005 % complete..
Get data 5.800000000000001 % complete..
Get data 6.0 % complete..
Get data 6.2 % complete..
Get data 6.4 % complete..
Get data 6.6000000000000005 % comp

In [38]:
df.to_csv("./fusion/data/tweet_temp4.csv", index=False)
# tweet_Temp는 '배민커넥터' 였음
# tweet_Temp2는 '배민'
# tweet_Temp3는 '배민커넥' 
# tweet_Temp4는 '식었음' --> 나이트 삐기 데이터..
 

In [39]:
tweet = pd.read_csv("./fusion/data/tweet_temp4.csv")
display(tweet)

,created,tweet_text
0,2020-11-20 07:08:51,RT @FF14_Orte: 근데 자만추 점점 클럽? 나이트? 분위기가 디폴트로 되어...
1,2020-11-20 05:05:33,"아ㅋㅋ 친구가 O딘 영상 보여줘서 보는데, 시네마틱 되게 잘뽑았다 디자인 좋다 이러..."
2,2020-11-20 04:25:14,RT @FF14_Orte: 근데 자만추 점점 클럽? 나이트? 분위기가 디폴트로 되어...
3,2020-11-20 04:23:58,RT @FF14_Orte: 근데 자만추 점점 클럽? 나이트? 분위기가 디폴트로 되어...
4,2020-11-20 04:05:56,RT @cumo_0: 졍우한테 돈세노 가사 따와서..\n잠깐 식었던 시간동안 그냥 ...
...,...,...
1480,2020-11-20 02:45:40,RT @FF14_Orte: 근데 자만추 점점 클럽? 나이트? 분위기가 디폴트로 되어...
1481,2020-11-20 02:33:03,RT @FF14_Orte: 근데 자만추 점점 클럽? 나이트? 분위기가 디폴트로 되어...
1482,2020-11-20 02:31:54,RT @FF14_Orte: 근데 자만추 점점 클럽? 나이트? 분위기가 디폴트로 되어...
1483,2020-11-20 02:29:10,RT @FF14_Orte: 근데 자만추 점점 클럽? 나이트? 분위기가 디폴트로 되어...


# 키워드 추출

In [ ]:
import re

# 텍스트 정제 함수 : 한글 이외의 문자는 전부 제거합니다.
def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글 처음부터 끝을 알리는 정규표현식
    result = hangul.sub('', text)
    return result

In [ ]:
# ‘tweet_text’ 피처에 이를 적용합니다.
tweet['ko_text'] = tweet['tweet_text'].apply(lambda x: text_cleaning(x))
tweet.tail()

In [ ]:
from konlpy.tag import Okt
from collections import Counter

# 한국어 약식 불용어사전 예시 파일입니다. 출처 - (https://www.ranks.nl/stopwords/korean)
korean_stopwords_path = "./fusion/data/korean_stopwords.txt"
with open(korean_stopwords_path, encoding='utf8') as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]

def get_nouns(x):
    nouns_tagger = Okt()
    nouns = nouns_tagger.nouns(x)
    
    # 한글자 키워드를 제거합니다.
    nouns = [noun for noun in nouns if len(noun) > 1]
    
    # 불용어를 제거합니다.
    nouns = [noun for noun in nouns if noun not in stopwords]
    
    return nouns

In [ ]:
# ‘ko_text’ 피처에 이를 적용합니다.
tweet['nouns'] = tweet['ko_text'].apply(lambda x: get_nouns(x))
print(df.shape)
tweet.tail()

In [ ]:
tweet.to_excel("./fusion/data/tweeter_result4.xlsx")

# 연관분석

In [ ]:
from apyori import apriori

# 장바구니 형태의 데이터(트랜잭션 데이터)를 생성
transactions = [
    ['쿠팡', '현실'],
    ['쿠리어', '소득'],
    ['배민', '쿠리어', '후기']
]

# 연관 분석을 수행
results = list(apriori(transactions))
for result in results:
    print(result)

In [ ]:
# 데이터 프레임 형태로 정리
columns = ['source', 'target', 'support']
network_tweet = pd.DataFrame(columns=columns)

# 규칙의 조건절을 source, 결과절을 target, 지지도를 support 라는 데이터 프레임의 피처로 변환
for result in results:
    if len(result.items) == 2:
        items = [x for x in result.items]
        row = [items[0], items[1], result.support]
        series = pd.Series(row, index=network_tweet.columns)
        network_tweet = network_tweet.append(series, ignore_index=True)

network_tweet.head()

In [ ]:
# 트랜잭션 데이터를 추출
transactions = tweet['nouns'].tolist()
#transactions = [transaction for transaction in transactions if transaction] # 공백 문자열을 방지합니다.
print(transactions[:10])

In [ ]:
# 연관 분석을 수행합니다.
# results = list(apriori(transactions,
#                         min_support=0.1))
# print(results) 

In [ ]:
# 데이터 프레임 형태로 정리
columns = ['source', 'target', 'support']
network_df = pd.DataFrame(columns=columns)

# 규칙의 조건절을 source, 결과절을 target, 지지도를 support 라는 데이터 프레임의 피처로 변환합니다.
for result in results:
    if len(result.items) == 2:
        items = [x for x in result.items]
        row = [items[0], items[1], result.support]
        series = pd.Series(row, index=network_df.columns)
        network_df = network_df.append(series, ignore_index=True)

network_df.head()

# 말뭉치로 추출

In [ ]:
tweet_corpus = "".join(tweet['ko_text'].tolist())
print(tweet_corpus)

In [ ]:
from konlpy.tag import Okt
from collections import Counter

# 명사 키워드를 추출
nouns_tagger = Okt()
nouns = nouns_tagger.nouns(tweet_corpus)
count = Counter(nouns)

# 한글자 키워드를 제거
remove_char_counter = Counter({x : count[x] for x in count if len(x) > 1})
print(remove_char_counter)

In [ ]:
# 위의 결과를 보기 쉽게 display
# 키워드와 키워드 빈도 점수를 ‘node’, ‘nodesize’ 라는 데이터 프레임의 피처로 생성
node_df = pd.DataFrame(remove_char_counter.items(), columns=['node', 'nodesize'])
node_df = node_df[node_df['nodesize'] >= 50] # 시각화의 편의를 위해 ‘nodesize’ 50 이하는 제거
node_df.head()

# 연관 키워드 네트워크 시각화

In [ ]:
#pip install networkx
import networkx as nx
plt.figure(figsize=(25,25))

# networkx 그래프 객체를 생성합니다.
G = nx.Graph()

# node_df의 키워드 빈도수를 데이터로 하여, 네트워크 그래프의 ‘노드’ 역할을 하는 원을 생성합니다.
for index, row in node_df.iterrows():
    G.add_node(row['node'], nodesize=row['nodesize'])
    
# network_df의 연관 분석 데이터를 기반으로, 네트워크 그래프의 ‘관계’ 역할을 하는 선을 생성합니다.
for index, row in network_df.iterrows():
    G.add_weighted_edges_from([(row['source'], row['target'], row['support'])])
    
# 그래프 디자인과 관련된 파라미터를 설정
pos = nx.spring_layout(G, k=0.6, iterations=50)
sizes = [G.nodes[node]['nodesize']*25 for node in G]
nx.draw(G, pos=pos, node_size=sizes)

# Windows 사용자는 AppleGothic 대신,'Malgun Gothic'. 그 외 OS는 OS에서 한글을 지원하는 기본 폰트를 입력합니다.
nx.draw_networkx_labels(G, pos=pos, font_family='Malgun Gothic', font_size=25)

# 그래프를 출력합니다.
ax = plt.gca()
plt.savefig("./fusion/data/네트워크 결과.png")
plt.show()


# 감성분석 따라해보기

In [ ]:
# 일단 참고해보자
# https://cyc1am3n.github.io/2018/11/10/classifying_korean_movie_review.html
# 감성분석은 테스트 데이터를 만들어 학습시켜야 함

In [ ]:
## 데이터 분석

In [ ]:
import pandas as pd

judge = pd.read_excel('./fusion/data/for_train.xlsx')
# display(judge)
judge_1 = judge['text']
# display(judge_1)
reality = pd.unique(judge_1)
df_real = pd.DataFrame(reality)
display(df_real)

In [ ]:
import pandas as pd

judge = pd.read_excel('./fusion/data/tweeter_result2.xlsx')
# display(judge)
judge_1 = judge['ko_text']
# display(judge_1)
reality = pd.unique(judge_1)
df_real = pd.DataFrame(reality)
display(df_real)

In [ ]:
import pandas as pd

judge = pd.read_excel('./fusion/data/tweeter_result3.xlsx')
# display(judge)
judge_1 = judge['ko_text']
# display(judge_1)
reality = pd.unique(judge_1)
df_real = pd.DataFrame(reality)
display(df_real)

In [ ]:
?tweepy

In [41]:
import pandas as pd

judge = pd.read_csv('./fusion/data/tweet_temp4.csv')
# display(judge)
judge_1 = judge['tweet_text']
# display(judge_1)
reality = pd.unique(judge_1)
df_real = pd.DataFrame(reality)
display(df_real)

,0
0,RT @FF14_Orte: 근데 자만추 점점 클럽? 나이트? 분위기가 디폴트로 되어...
1,"아ㅋㅋ 친구가 O딘 영상 보여줘서 보는데, 시네마틱 되게 잘뽑았다 디자인 좋다 이러..."
2,RT @cumo_0: 졍우한테 돈세노 가사 따와서..\n잠깐 식었던 시간동안 그냥 ...
3,화장실에서 이 짓 하다가 **하는 들신을 방금 생각했는데 .. .. . **하고 ...
